In [1]:
from spatialoperations import SpatialOperations
from spatialoperations.config import SpatialOpsConfig


config = SpatialOpsConfig(
    root="s3://cl-lake-test/sids-test-tough-topobathy",
    dx=0.00026977476123492776,
    epsg=4326,
    bounds=[-179.99, -50, 179.99, 50],
    chunk_size=5000,
)
so = SpatialOperations(config=config)
so

In [2]:
import geopandas as gpd
import numpy as np

countries = [
    "FSM_10",
    # "FSM_05",
    # "FSM_08",
    # "FSM_21",
    # "FSM_20",
    # "FSM_18",
    # "FSM_11",
    # "KIR_09"
]
gdf = gpd.read_file("/cccr-lab/001_projects/002_nbs-adapt/003_regions_study_units/All_Regions_Marina/02_Polygons_SIDS_20250701/Polygons_SIDS_20250701_final.gpkg")   
regions = gdf[
    gdf["Subreg_Name"].apply(lambda x: np.any([c in x for c in countries]))
]
regions











,region_id,HAS_REEFS,NumberOfCe,PolygonAre,ACADataAva,ACABenthic,ACABathyme,utmzone,utmcode,Subreg_Name,geometry
52,53,NaN,4.193852e+06,3.774467e+09,YES,YES,YES,56PN,32656,FSM_10,"MULTIPOLYGON (((150.21069 9.05281, 150.53484 8..."


In [3]:
from glob import glob
from spatialoperations.types.xrtypes import RasterLike
import os
import pandas as pd
from shapely.geometry import box
import rioxarray as rxr

idxs=so.ro.zarr_manager.chunks_initialized(
    "ACA",
    "topobathy/bathymetry"
)
fabdem = glob('/cccr-lab/000_data-repository/011_GEBCO/GEBCO_2024/*.tif')

def path_to_bounds(path):
    ds = rxr.open_rasterio(path)
    return box(*ds.rio.bounds())

df = pd.DataFrame(fabdem, columns=["path"])
df["bounds"] = df["path"].apply(path_to_bounds)
bounds_gdf = gpd.GeoDataFrame(df, geometry="bounds", crs=4326)
bounds_gdf.head()

joined = gpd.sjoin(bounds_gdf, regions, how="inner")
paths = joined["path"].unique()
paths

array(['/cccr-lab/000_data-repository/011_GEBCO/GEBCO_2024/gebco_2024_n90.0_s0.0_w60.0_e180.0.tif'],
      dtype=object)

In [4]:
import rioxarray as rxr
import geopandas as gpd
import os
from tqdm import tqdm
from glob import glob
from spatialoperations.types.xrtypes import RasterLike
import xarray as xr

def safe_get_region(path):
    try:
        ds = rxr.open_rasterio(path).isel(band=0)
        init_crs = ds.rio.crs
        ds = xr.where(ds < 0, ds, np.nan)
        ds.rio.write_crs(init_crs, inplace=True)
        return RasterLike(ds, name=path.split('/')[-1].split('.')[0])
    except Exception as e:
        return None
        
GEBCO = {
    f"idx_{idx}": safe_get_region(path)
    for idx, path in tqdm(enumerate(paths))
}

GEBCO = {
    k: v for k, v in GEBCO.items() if v is not None
}

GEBCO
# FABDEM.keys()


1it [00:05,  5.06s/it]


{'idx_0': RasterLike(_data=<xarray.DataArray 'elevation' (y: 21600, x: 28800)> Size: 5GB
 array([[-4236., -4236., -4236., ..., -4223., -4223., -4222.],
        [-4235., -4235., -4236., ..., -4223., -4222., -4221.],
        [-4234., -4235., -4236., ..., -4222., -4221., -4220.],
        ...,
        [-4564., -4572., -4577., ..., -5248., -5253., -5255.],
        [-4563., -4571., -4577., ..., -5251., -5257., -5259.],
        [-4563., -4570., -4576., ..., -5250., -5256., -5259.]])
 Coordinates:
     band         int64 8B 1
   * x            (x) float64 230kB 60.0 60.01 60.01 60.01 ... 180.0 180.0 180.0
   * y            (y) float64 173kB 90.0 89.99 89.99 ... 0.01042 0.00625 0.002083
     crs          int64 8B 0
     spatial_ref  int64 8B 0, name='gebco_2024_n90')}

In [6]:
from spatialoperations import rasterops
from spatialoperations.rasterops import intake
from spatialoperations import compute
import rasterio

jlc = compute.JoblibCompute(n_jobs=20)

so.ro.intake.import_multi_raster_to_zarr_with_prep(
    GEBCO, 
    "topobathy/bathymetry", 
    idxs=idxs,
    output_path="GEBCO",
    compute=jlc,
    reproject_match_kwargs={"reprojection_method": rasterio.enums.Resampling.cubic}
)

100%|██████████████████████████████████████| 10/10 [00:00<00:00, 142663.40it/s]
